In [38]:
import requests
from bs4 import BeautifulSoup
import csv
import string
import pandas as pd

In [2]:
base_url = "https://www.abjjad.com/books/220759001/%D8%B1%D9%88%D8%A7%D9%8A%D8%A7%D8%AA-%D9%88%D9%82%D8%B5%D8%B5"
main_website = "https://www.abjjad.com"
categories_page_url = "https://www.abjjad.com/books/categories"

In [3]:
#loading the page and change it to soup
category_page = requests.get(categories_page_url)
category_page_src = category_page.content
category_page_soup = BeautifulSoup(category_page_src, "lxml")

In [ ]:
category_page_div = category_page_soup.findAll("div",{"class":"category-list-width"})

In [ ]:
category_page_div

In [6]:
list_of_categories_links = []

In [7]:
for items in category_page_div:
    list_of_categories_links.append(items.find("a").get("href"))
    print(items.find("a").text)

print(list_of_categories_links)

روايات وقصص  > 
كتب الأدب  > 
كتب تاريخية  > 
كتب دينية  > 
كتب سياسية  > 
كتب علمية  > 
مال وأعمال  > 
علم النفس وتطوير الذات  > 
السيرة الذاتية والمذكرات  > 
فلسفة  > 
لغات  > 
قانون  > 
تكنولوجيا وإنترنت  > 
الصحافة والإعلام  > 
الطب والصحة  > 
الأسرة والطفل  > 
الرياضة والتسلية  > 
فنون  > 
كتب أطفال  > 
السفر والترحال  > 
ميثولوجيا وأساطير  > 
مراجع وأبحاث  > 
['/books/220759001/%D8%B1%D9%88%D8%A7%D9%8A%D8%A7%D8%AA-%D9%88%D9%82%D8%B5%D8%B5', '/books/220692594/%D9%83%D8%AA%D8%A8-%D8%A7%D9%84%D8%A7%D8%AF%D8%A8', '/books/220692565/%D9%83%D8%AA%D8%A8-%D8%AA%D8%A7%D8%B1%D9%8A%D8%AE%D9%8A%D8%A9', '/books/220692661/%D9%83%D8%AA%D8%A8-%D8%AF%D9%8A%D9%86%D9%8A%D8%A9', '/books/220692719/%D9%83%D8%AA%D8%A8-%D8%B3%D9%8A%D8%A7%D8%B3%D9%8A%D8%A9', '/books/220692682/%D9%83%D8%AA%D8%A8-%D8%B9%D9%84%D9%85%D9%8A%D8%A9', '/books/220692522/%D9%85%D8%A7%D9%84-%D9%88%D8%A7%D8%B9%D9%85%D8%A7%D9%84', '/books/220692776/%D8%B9%D9%84%D9%85-%D8%A7%D9%84%D9%86%D9%81%D8%B3-%D9%88%D8%AA%D8%B7%D9%88%D9%8A%D8%B1-

In [8]:
list_of_categories_links = list(set(list_of_categories_links))

In [9]:
len(list_of_categories_links)

22

In [10]:
book_title = []
book_author = []
book_description = []
book_cover_photo_url = []
book_url = []
book_number_of_pages = []
book_rating = []
book_genera = []
book_numbers_of_reviews = []


In [11]:
#in this loop we will loops on the books of each category and extract the data from them
#now this loop retrive each link for all books tomorrow insha'a allah i will begin to retrive data from these pages
for items in list_of_categories_links:
    category_page_url = f"{main_website}/{items}"
    try:
        page_link = requests.get(category_page_url, timeout=10)
        page_link.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {category_page_url}: {e}")
        continue
    page_src = page_link.content
    page_soup = BeautifulSoup(page_src, "lxml")
    genre = page_soup.find("h1").text.strip()
    all_books_count = page_soup.find("span",{"class":"resultcount"}).text.strip()
    all_books_count = int(all_books_count)
    all_books_divs = page_soup.findAll("div",{"class":"thebookbadge"})
    number_of_iteration = all_books_count // len(all_books_divs)
    print(f"i have now get the{genre}category")
    for i in range(number_of_iteration):
      category_page_url = f"{main_website}/{items}/{i}"
      try:
          page_link = requests.get(category_page_url, timeout=10)
          page_link.raise_for_status()
      except requests.exceptions.RequestException as e:
          print(f"Error fetching {category_page_url}: {e}")
          continue
      page_src = page_link.content
      page_soup = BeautifulSoup(page_src, "lxml")
      all_books_divs = page_soup.findAll("div",{"class":"thebookbadge"})
      for div in all_books_divs:
        #all_books_links.append(div.find("a").get("href"))
        book_link = f"{main_website}/{div.find('a').get('href')}"
        try:
            book_page = requests.get(book_link, timeout=10)
            book_page.raise_for_status()
        except requests.exceptions.RequestException as e:
              print(f"Error fetching {book_link}: {e}")
              continue
        book_page_src = book_page.content
        book_page_soup = BeautifulSoup(book_page_src, "lxml")
        print(f"iam now scraping this book {book_link}")
        book_title.append(book_page_soup.find("h1").text.strip() if book_page_soup.find("h1") else None)
        book_author.append(book_page_soup.find("a",{"itemprop":"author"}).text.strip() if book_page_soup.find("a",{"itemprop":"author"}) else None)
        book_description.append(book_page_soup.find("span",{"class":"content"}).text.strip() if book_page_soup.find("span",{"class":"content"}) else None)
        book_cover_photo_url.append(book_page_soup.find("div",{"class":"img editionphoto"}).find("a").get("href") if book_page_soup.find("div",{"class":"img editionphoto"}) else None)
        book_url.append(book_link)
        ul_element = book_page_soup.find("ul", {"class": "rowlist"})
        if ul_element:
          li_elements = ul_element.findAll("li")
          book_number_of_pages.append(li_elements[1].text.strip() if len(li_elements) > 1 else None)
        else:
          book_number_of_pages.append(None)
        book_rating.append(book_page_soup.find("span",{"class":"number"}).text.strip() if book_page_soup.find("span",{"class":"number"}) else None)
        book_genera.append(genre)
        book_numbers_of_reviews.append(book_page_soup.find("span",{"itemprop":"ratingCount"}).text.strip() if book_page_soup.find("span",{"itemprop":"ratingCount"}) else None)
        print("i have done this book")





<ipython-input-11-62a8385875d2>:16: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  all_books_divs = page_soup.findAll("div",{"class":"thebookbadge"})


i have now get theالأسرة والطفلcategory


<ipython-input-11-62a8385875d2>:29: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  all_books_divs = page_soup.findAll("div",{"class":"thebookbadge"})


iam now scraping this book https://www.abjjad.com//book/2791669774/%D9%83%D9%84%D8%A7%D9%85-%D8%A7%D9%84%D8%B9%D9%84%D9%85-%D9%81%D9%8A-%D8%A7%D9%84%D8%AD%D8%A8-%D9%88%D8%A7%D9%84%D8%AC%D9%86%D8%B3/2792718350
i have done this book


<ipython-input-11-62a8385875d2>:49: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  li_elements = ul_element.findAll("li")


Streaming output truncated to the last 5000 lines.
i have done this book
iam now scraping this book https://www.abjjad.com//book/2783084545/%D9%81%D9%84%D8%B3%D9%81%D8%A9-%D8%A7%D9%84%D8%BA%D8%B2%D8%A7%D9%84%D9%8A/2784133121
i have done this book
iam now scraping this book https://www.abjjad.com//book/2784428035/%D8%A7%D8%A8%D9%88-%D8%A8%D9%83%D8%B1-%D8%B1%D8%B6%D9%8A-%D8%A7%D9%84%D9%84%D9%87-%D8%B9%D9%86%D9%87-%D8%B1%D9%85%D8%B2-%D8%A7%D9%84%D8%A7%D9%8A%D9%85%D8%A7%D9%86-%D9%88%D8%A7%D9%84%D8%AB%D8%A8%D8%A7%D8%AA-%D8%B3%D9%84%D8%B3%D9%84/2785476611
i have done this book
iam now scraping this book https://www.abjjad.com//book/2792161288/%D8%B3%D9%8A%D8%AF-%D9%82%D8%B7%D8%A8-%D9%88%D8%AB%D9%88%D8%B1%D8%A9-%D9%8A%D9%88%D9%84%D9%8A%D9%88/2793209864
i have done this book
iam now scraping this book https://www.abjjad.com//book/2116485120/%D8%B9%D8%A8%D9%82%D8%B1%D9%8A%D8%A9-%D8%A7%D9%84%D8%A7%D9%85%D8%A7%D9%85-%D8%B9%D9%84%D9%8A/2116616192
i have done this book
iam now scraping this book ht

In [12]:
len(book_title)

31157

In [13]:
len(book_author)

31157

In [14]:
len(book_description)

31157

In [15]:
len(book_cover_photo_url)

31157

In [16]:
len(book_url)

31157

In [17]:
len(book_number_of_pages)

31157

In [18]:
len(book_rating)

31157

In [19]:
len(book_genera)

31157

In [20]:
len(book_numbers_of_reviews)

31157

In [23]:
#this is used to fill the data not row by row but to take the information of each book and put it in one row
from itertools import zip_longest

In [ ]:
#this is explain for what the zip_longest do
"""
list_1 = [1,2,3]
list_2 = ["a","b","c"]
list_3 = [list_1,list_2]
zip_longest(*list_3)--->>>[(1, 'a'), (2, 'b'), (3, 'c')]
"""

In [27]:
book_lists = [book_title, book_author, book_description, book_number_of_pages, book_rating, book_genera, book_numbers_of_reviews, book_url, book_cover_photo_url]
exported_data = zip_longest(*book_lists)

In [28]:
#now we will add the data to a csv file and save the csv file
with open("books_data.csv", "w", encoding="utf-8-sig",newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["book_title", "book_author", "book_description", "book_number_of_pages", "book_rating", "book_genera", "book_numbers_of_reviews", "book_url", "book_cover_photo_url"])
    csv_writer.writerows(exported_data)


In [ ]:
#we then created a dataframe to save the file in another format which is xlsx
df = pd.DataFrame({
    "book_title": book_title,
    "book_author": book_author,
    "book_description": book_description,
    "book_number_of_pages": book_number_of_pages,
    "book_rating": book_rating,
    "book_genera": book_genera,
    "book_numbers_of_reviews": book_numbers_of_reviews,
    "book_url": book_url,
    "book_cover_photo_url": book_cover_photo_url
})

In [37]:
df.to_excel("books_data.xlsx", index=False, engine='openpyxl')